# IBM Data Science Capstone Project

**Hello and Welcome!**

This Notebook will be used solely for IBM Data Science Capstone Project! I wish a great day and a lot of luck reviewing my results!
<br/>
Below you can find couple of my favourite jokes about being Data Scientist to lighten your mood:

___

>*The data science motto: If at first you don’t succeed; call it version 1.0*

<!-- -->
___

>*Why should you take a data scientist with you into the jungle? <br/>- They can take care of Python problems*

___

In [105]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Assignment #2: Segmenting and Clustering Neighborhoods in Toronto

The main task is to explore and cluster neihborhoods in Toronto, Canada, using Foresquare API and K-means clustring algorithm.

**Part 1**<br/>
Scrape Neighborhood list from wikipedia.

In [106]:
# Importing scraping libraries
import requests
from bs4 import BeautifulSoup as bs4

In [107]:
# Defining url and requesting source HTML
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)

In [108]:
# Souping the html :D
soup = bs4(r.content,'html')

In [109]:
# Find table by class name and convert to DataFrame
table = soup.find_all('table',{'class':'wikitable sortable'})
neighbrhd_df = pd.read_html(str(table[0]))[0]
neighbrhd_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [110]:
# Filtering DataFrame
neighbrhd_df = neighbrhd_df[neighbrhd_df.Neighbourhood != 'Not assigned']
neighbrhd_df.columns = ['PostalCode','Borough','Neighborhood']
neighbrhd_df.reset_index(drop=True, inplace=True)
neighbrhd_df.head()

#NOTE: As you can see below Wikipedia already grouped Neighborhoods by postal codes (ex. M5A: "Regent Park, Harbourfront", 
# Hence, no addtional grouping required. 

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [111]:
neighbrhd_df.shape

(103, 3)

**Part 2**<br/>
Retrieving latitude, longitude for each naighborhood using geocoder. 

In [112]:
# Google Goecoder did not worked for me, I used csv attached to task
geo_df = pd.read_csv('Geospatial_Coordinates.csv')
geo_df.columns = ['PostalCode','Latitude','Longitude']
geo_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [113]:
# Example
postal_code = 'M5G'
geo_df[geo_df['PostalCode'] == postal_code]

,PostalCode,Latitude,Longitude
57,M5G,43.657952,-79.387383


In [114]:
neighbrhd_df = pd.merge(neighbrhd_df, geo_df, on='PostalCode', how='left')
neighbrhd_df.shape

(103, 5)

In [115]:
neighbrhd_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [116]:
import folium

latitude, longitude = 43.6532,-79.3832
# create map of Toronto using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighbrhd_df['Latitude'], neighbrhd_df['Longitude'], neighbrhd_df['Borough'], neighbrhd_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [117]:
print('Thank you for your time!')

Thank you for your time!
